In [1]:
import pandas as pd
import json
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn import preprocessing
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
pstemmer = PorterStemmer()

In [3]:
jsonfile = r'.\Data\train\train.json'
with open(jsonfile) as train_json:
    json_dict = json.load(train_json)

In [4]:
sno = []
cuisine = []
ingredients = []
for i in range(len(json_dict)):
    sno.append(json_dict[i]['id'])
    cuisine.append(json_dict[i]['cuisine'])
    ingredients.append(json_dict[i]['ingredients'])

In [5]:
df = pd.DataFrame({'id':sno, 
                   'cuisine':cuisine, 
                   'ingredients':ingredients})

In [6]:
def analyzer(df):
    s_df = []
    for s in df:
        s = ' '.join(s)
        s_df.append(s) 
    import re
    mod_df=[]
    for s in s_df:
        s=re.sub(r'[^\w\s]','',s)
        s=re.sub(r"(\d)", "", s)    
        s=re.sub(r'\([^)]*\)', '', s)
        s=re.sub(u'\w*\u2122', '', s)
        s=s.lower()    
        #Remove Stop Words
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(s)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        s=' '.join(filtered_sentence)    
        #Porter Stemmer Algorithm
        words = word_tokenize(s)
        word_ps=[]
        for w in words:
           word_ps.append(pstemmer.stem(w))
        s=' '.join(word_ps)
        mod_df.append(s)
    return mod_df

In [7]:
def analyzer1(df):
    import re
    mod_df=[]
    for s in df:
        s=re.sub(r'[^\w\s]','',s)
        s=re.sub(r"(\d)", "", s)
        s=re.sub(r'\([^)]*\)', '', s)   
        s=re.sub(u'\w*\u2122', '', s)
        s=s.lower()
        #Remove Stop Words
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(s)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        s=' '.join(filtered_sentence)
        #Porter Stemmer Algorithm
        words = word_tokenize(s)
        word_ps=[]
        for w in words:
           word_ps.append(pstemmer.stem(w))
        s=' '.join(word_ps)
        mod_df.append(s)
    return mod_df

In [8]:
df['ingredients_mod']= analyzer(df['ingredients'])

In [9]:
mylist =['fish', 'goat', 'chicken','beef','pork','prawn','egg','Katsuobushi','mackrel','fillet','lamb','steak','salmon','shrimp','bacon','ham']
pattern = '|'.join(mylist)
df['veg']=df.ingredients_mod.str.contains(pattern) 
df.loc[df.veg == True,'veg'] = 'non-vegetarian'
df.loc[df.veg == False,'veg'] = 'vegetarian'

In [10]:
file = r'.\Data\test\test.csv'
df_test = pd.read_csv(file,encoding='ISO-8859-1')
df_test['ingredients_mod']= analyzer1(df_test['ingredients'])

In [11]:
vectorizer = TfidfVectorizer()
ingredient_train = vectorizer.fit_transform(df['ingredients_mod'])
ingredient_test = vectorizer.transform(df_test['ingredients_mod'])

In [12]:
elable = preprocessing.LabelEncoder()
elable.fit(df['cuisine'])
df['cuisine']=elable.transform(df['cuisine'])
elable.fit(df['veg'])
df['veg']=elable.transform(df['veg'])

In [13]:
veg_map={'0':'non-vegetarian', '1':'vegetarian'}
cuisine_map={'0':'brazilian', '1':'british', '2':'cajun_creole', '3':'chinese', '4':'filipino', '5':'french', '6':'greek', '7':'indian', '8':'irish', '9':'italian', '10':'jamaican', '11':'japanese', '12':'korean', '13':'mexican', '14':'moroccan', '15':'russian', '16':'southern_us', '17':'spanish', '18':'thai', '19':'vietnamese'}

In [14]:
Cuisine_train=[]
Type_train=[]
Cuisine_train = df['cuisine']
Type_train = df['veg']

In [15]:
lin_cuisine = svm.LinearSVC(C=1)
lin_veg = svm.LinearSVC(C=1)
lin_cuisine.fit(ingredient_train, Cuisine_train)
lin_veg.fit(ingredient_train, Type_train)
Cuisine_pred_idx=lin_cuisine.predict(ingredient_test)
Type_pred_idx=lin_veg.predict(ingredient_test)

In [29]:
Cuisine_pred=[]
for i in range(len(Cuisine_pred_idx)):
    Cuisine_pred.append(cuisine_map[str(Cuisine_pred_idx[i])])
       
Type_pred=[]
for i in range(len(Type_pred_idx)):
    Type_pred.append(veg_map[str(Type_pred_idx[i])])    

df_test['cuisine'] =  Cuisine_pred
df_test['Veg-NonVeg'] =  Type_pred
Result = pd.DataFrame({'Receipe Image URL':df_test['receipe-image-src'],'Receipes':df_test['receipes'], 'Cuisine':df_test['cuisine'], 'Veg-NonVeg':df_test['Veg-NonVeg'],'Cook Time':df_test['cook time'],'Effort':df_test['effort'],'Ingredients':df_test['ingredients'],'Cooking Method':df_test['receipe-method'],'Receipes URL':df_test['receipes-href']})
print(Result)

                                    Receipe Image URL  \
0   //www.bbcgoodfood.com/sites/default/files/styl...   
1   //www.bbcgoodfood.com/sites/default/files/styl...   
2   //www.bbcgoodfood.com/sites/default/files/styl...   
3   //www.bbcgoodfood.com/sites/default/files/styl...   
4   //www.bbcgoodfood.com/sites/default/files/styl...   
5   //www.bbcgoodfood.com/sites/default/files/styl...   
6   //www.bbcgoodfood.com/sites/default/files/styl...   
7   //www.bbcgoodfood.com/sites/default/files/styl...   
8   //www.bbcgoodfood.com/sites/default/files/styl...   
9   //www.bbcgoodfood.com/sites/default/files/styl...   
10  //www.bbcgoodfood.com/sites/default/files/styl...   
11  //www.bbcgoodfood.com/sites/default/files/styl...   
12  //www.bbcgoodfood.com/sites/default/files/styl...   
13  //www.bbcgoodfood.com/sites/default/files/styl...   
14  //www.bbcgoodfood.com/sites/default/files/styl...   
15  //www.bbcgoodfood.com/sites/default/files/styl...   
16  //www.bbcgoodfood.com/sites

In [30]:
Result.to_csv("output.csv", index=False)